In [4]:
##do_runcode
##%overwritefile
##%file:src/do_c_runcode.py
##%noruncode
    def do_runcode(self,return_code,fil_ename,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=return_code
        fil_ename=fil_ename
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''
        ##代码运行前
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,1)
        if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        self._write_to_stdout("The process :"+fil_ename+"\n")
        ################# repl mode run code files
        #FIXME:
        if magics['runmode']=='repl':
            self._start_replprg(fil_ename,magics['args'],magics)

            return_code=self.replwrapper.child.status
            bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,2)
            # if bcancel_exec:return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
            
            return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        ############################################

    ############################################
        #################dynamically load and execute code
        #FIXME:
        if len(magics['dlrun'])>0:
            p = self.create_jupyter_subprocess([self.master_path, fil_ename] + magics['args'],env=self.addkey2dict(magics,'env'))
        #################
        else:
            p = self.create_jupyter_subprocess([fil_ename] + magics['args'],env=self.addkey2dict(magics,'env'))
        self.subprocess=p
        self.g_rtsps[str(p.pid)]=p
        return_code=p.returncode
        ##代码启动后
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,2)
        if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        
        if len(self.addkey2dict(magics,'showpid'))>0:
            self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
        while p.poll() is None:
            p.write_contents(magics)
        
        ############################################
        p.write_contents(magics)
        # wait for threads to finish, so output is always shown
        p._stdout_thread.join()
        p._stderr_thread.join()
        p.write_contents(magics)

        return_code=p.returncode
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,3)
        if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        ##代码运行结束
        # now remove the files we have just created
        # if(os.path.exists(source_file.name)):
            # os.remove(source_file.name)
        # if(os.path.exists(binary_filename)):
            # os.remove(binary_filename)

        # if p.returncode != 0:
            # self._write_to_stderr("[C kernel] Executable exited with code {}".format(p.returncode))
        return bcancel_exec,retinfo,magics, code,fil_ename,retstr

[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/C/src/do_c_runcode.py created successfully


In [5]:
##do_compile_code
##%overwritefile
##%file:src/do_c_compilecode.py
##%noruncode
    def do_compile_code(self,return_code,fil_ename,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=0
        fil_ename=fil_ename
        sourcefilename=fil_ename
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''
        # Generate executable file :being
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,2,1)
        if bcancel_exec:return  bcancel_exec,retinfo,magics, code,fil_ename,retstr

        if len(self.addkey2dict(magics,'file'))>0:
            fil_ename=magics['file'][0]
        # else: fil_ename=sourcefilename
        returncode,binary_filename=self._exec_gcc_(fil_ename,magics)
        fil_ename=binary_filename
        return_code=returncode
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,2,2)
        if bcancel_exec:return  bcancel_exec,self.get_retinfo()
        
        if returncode!=0:return  True,retinfo, code,fil_ename,retstr
        # Generate executable file :end
        return bcancel_exec,retinfo,magics, code,fil_ename,retstr

[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/C/src/do_c_compilecode.py created successfully


In [6]:
##do_create_codefile
##%overwritefile
##%file:src/do_c_create_codefile.py
##%noruncode
    def do_create_codefile(self,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=0
        fil_ename=''
        sourcefilename=''
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,1,1)
        if bcancel_exec:return  bcancel_exec,self.get_retinfo(),magics, code,fil_ename,retstr

        with self.new_temp_file(suffix='.c') as source_file:
            source_file.write(code)
            source_file.flush()
            sourcefilename=source_file.name 
            newsrcfilename=source_file.name 
            fil_ename=newsrcfilename
            return_code=True
            # Generate new src file
            bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,1,2)
            if bcancel_exec:return  bcancel_exec,self.get_retinfo(),magics, code,fil_ename,retstr

            # if len(self.addkey2dict(magics,'onlycsfile'))>0:
            #     if len(self.addkey2dict(magics,'file'))<1:
            #         self._log("no file name parameter\n",2)
            #     return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
            ##编译代码文件
            bcancel_exec,retinfo,magics, code,fil_ename,retstr=self.do_compile_code(return_code,fil_ename,magics,code, silent, store_history=store_history,
                    user_expressions=user_expressions, allow_stdin=allow_stdin)
            if bcancel_exec:return  bcancel_exec,self.get_retinfo(),magics, code,fil_ename,retstr
        ############# only run gcc，no not run executable file
        if len(self.addkey2dict(magics,'onlyrungcc'))>0:
            self._log("only run gcc \n")
        return  bcancel_exec,self.get_retinfo(),magics, code,fil_ename,retstr

[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/C/src/do_c_create_codefile.py created successfully


In [7]:
##do_preexecute
##%overwritefile
##%file:src/do_c_preexecute.py
##%noruncode
    def do_preexecute(self,code,magics, silent, store_history=True,
                user_expressions=None, allow_stdin=False):
        bcancel_exec=False
        retinfo=self.get_retinfo()
        ##扫描代码
        # magics, code = self._filter_magics(code)
        # magics, code = self.mag.filter(code)
        if len(self.addkey2dict(magics,'replcmdmode'))>0:
            bcancel_exec=True
            retinfo= self.send_replcmd(code, silent, store_history=store_history,
                user_expressions=user_expressions, allow_stdin=allow_stdin)
            return bcancel_exec,retinfo,magics, code
        ############# run gdb and send command begin
        if len(magics['rungdb'])>0:
            bcancel_exec=True
            retinfo= self.replgdb_sendcmd(code,silent, store_history,
                user_expressions, allow_stdin)
            return bcancel_exec,retinfo,magics, code
        ############# run gdb and send command
        #############send replcmd's command
        if magics['runmode']=='repl':
            if hasattr(self, 'replcmdwrapper'):
                if self.replcmdwrapper :
                    bcancel_exec=True
                    retinfo= self.repl_sendcmd(code, silent, store_history,
                        user_expressions, allow_stdin,magics)
                    return bcancel_exec,retinfo,magics, code
        #FIXME:
        #############send replcmd's command end
        ############# only run command mark
        
        if len(self.addkey2dict(magics,'onlyruncmd'))>0:
            return bcancel_exec,retinfo,magics, code
        ############# only create source file
        
        if len(self.addkey2dict(magics,'onlycsfile'))<1 :
            magics, code = self._add_main(magics, code)
        return bcancel_exec,retinfo,magics, code

[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/C/src/do_c_preexecute.py created successfully


In [4]:
## new kernel.py
##%overwritefile
##%file:../../../jupyter-MyC-kernel/jupyter_MyC_kernel/kernel.py
###%file:c_kernel.py
##%noruncode
#
#   MyC Jupyter Kernel 
#   generated by MyPython
#
##%include:../../src/head.py
##%include:../../src/common.py
##%include:../../src/Mykernel.py
class CKernel(MyKernel):
    implementation = 'jupyter-MyC-kernel'
    implementation_version = '1.0'
    language = 'C'
    language_version = 'C11'
    language_info = {'name': 'text/x-csrc',
                     'mimetype': 'text/x-csrc',
                     'file_extension': '.c'}
    runfiletype='exe'
    banner = "C kernel.\n" \
             "Uses gcc, compiles in C11, and creates source code files and executables in temporary folder.\n"

    main_head = "#include <stdio.h>\n" \
            "#include <math.h>\n" \
            "int main(int argc, char* argv[], char** env){\n"

    main_foot = "\nreturn 0;\n}"



    def __init__(self, *args, **kwargs):
        super(CKernel, self).__init__(*args, **kwargs)
        # MyKernel.__init__(self, cmd_or_spawn, orig_prompt,
        #                               prompt_change,extra_init_cmd=extra_init_cmd)
        self.kernelinfo='[MyC Kernel]'
        self._allow_stdin = True
        self.readOnlyFileSystem = False
        self.bufferedOutput = True
        self.linkMaths = True # always link math library
        self.wAll = True # show all warnings by default
        self.wError = False # but keep comipiling for warnings
        
#################
##%include:src/compile_with_gcc.py
##%include:src/c_exec_gcc_.py
##%include:src/c_start_gdb.py
##%include:src/c_do_replexecutegdb.py
##%include:src/c_replgdb_sendcmd.py

##%include:src/do_c_runcode.py
##%include:src/do_c_compilecode.py
##%include:src/do_c_create_codefile.py
##%include:src/do_c_preexecute.py



[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/C/../../../jupyter-MyC-kernel/jupyter_MyC_kernel/kernel.py created successfully
